# Download data and process interferogram stack
George Brencher

## Create and configure environment

In [2]:
import os
if not os.path.exists('/srv/conda/envs/isce2'):
    !mamba create -n isce2 --file https://github.com/UW-GDA/Friendly-InSAR-time-series/blob/main/conda-linux-64.lock
    !conda activate isce2
    !python -m ipykernel install --user --name=isce2

Make sure kernel is set to 'isce2' or 'Python [conda env:isce2]'

'topStack' is a Sentinel 1 interferogram stack processing workflow which has been contributed to isce. To get access to topsStack, which does not come with the conda install of isce, we need to clone isce2 from github. 

In [2]:
#!git clone https://github.com/isce-framework/isce2.git

Cloning into 'isce2'...
remote: Enumerating objects: 8344, done.
remote: Counting objects: 100% (835/835), done.
remote: Compressing objects: 100% (472/472), done.
remote: Total 8344 (delta 444), reused 634 (delta 355), pack-reused 7509
Receiving objects: 100% (8344/8344), 7.41 MiB | 23.36 MiB/s, done.
Resolving deltas: 100% (5006/5006), done.


In [3]:
import isce

# Set Environment variables so that you can call ISCE from the command line
os.environ['ISCE_HOME'] = os.path.dirname(isce.__file__)
os.environ['ISCE_ROOT'] = os.path.dirname(os.environ['ISCE_HOME'])
os.environ['PATH']+='{ISCE_HOME}/bin:{ISCE_HOME}/applications'.format(**os.environ)

# Set path to topsStack workflow in isce source directory
CWD = os.getcwd()
# May need to change relative path to topsStack on your system
os.environ['PATH']+=f'/../../../../../../../../..{CWD}/isce2/contrib/stack/topsStack'
print(os.environ['PATH'])

/srv/conda/envs/notebook/bin:/srv/conda/condabin:/srv/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin/srv/conda/envs/isce2/lib/python3.9/site-packages/isce/bin:/srv/conda/envs/isce2/lib/python3.9/site-packages/isce/applications/../../../../../../../../../home/jovyan/Friendly-InSAR-time-series/isce2/contrib/stack/topsStack


In [4]:
# Make sure stack processing scripts are in path
!which stackSentinel.py

/srv/conda/envs/isce2/lib/python3.9/site-packages/isce/applications/../../../../../../../../../home/jovyan/Friendly-InSAR-time-series/isce2/contrib/stack/topsStack/stackSentinel.py


In [ ]:
# Do downloads and processing in a temporary folder to avoid clogging the disk
os.chdir('/tmp')

## Download data

In [7]:
#check disk space prior to downloading SLCs
!df -h ~

Filesystem      Size  Used Avail Use% Mounted on
/dev/sdd         15G  193M   15G   2% /home/jovyan


In [43]:
import getpass
import asf_search as asf
import logging

# Update this with your NASA Earthdata login to download SLC data
EARTHDATA_LOGIN = "qbrencherUW"
EARTHDATA_PASSWORD = getpass.getpass()

# prevent DEBUG messages
logging.getLogger("urllib3").setLevel(logging.WARNING)

try:
    user_pass_session = asf.ASFSession().auth_with_creds(EARTHDATA_LOGIN, EARTHDATA_PASSWORD)
except asf.ASFAuthenticationError as e:
    print(f'Auth failed: {e}')
else:
    print('Success!')

 ········


2022-03-05 17:43:30,107 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): urs.earthdata.nasa.gov:443
2022-03-05 17:43:30,527 - urllib3.connectionpool - DEBUG - https://urs.earthdata.nasa.gov:443 "GET /oauth/authorize?client_id=BO_n7nTIlMljdvU6kRRB3g&response_type=code&redirect_uri=https://auth.asf.alaska.edu/login HTTP/1.1" 302 None
2022-03-05 17:43:30,534 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): auth.asf.alaska.edu:443
2022-03-05 17:43:31,623 - urllib3.connectionpool - DEBUG - https://auth.asf.alaska.edu:443 "GET /login?code=60b503edfa5ed8143b47f3cb519e00e1a63ce4092799ce506bda8927cce89616 HTTP/1.1" 301 0
2022-03-05 17:43:32,912 - urllib3.connectionpool - DEBUG - https://auth.asf.alaska.edu:443 "GET / HTTP/1.1" 200 244717
Success!


In [22]:
# Set area of interest with two corner points of a box [lat, lon]
point1 = [27.875165, 86.892988]
point2 = [27.918111, 86.949540]

aoi = f'POLYGON(({point1[1]} {point1[0]}, {point1[1]} {point2[0]}, {point2[1]} {point2[0]}, {point2[1]} {point1[0]}, {point1[1]} {point1[0]}))'

In [39]:
# Set platform, start and end dates, flight direction, and processing level
opts = {'platform': asf.PLATFORM.SENTINEL1, 
        'start': '2021-08-01T00:00:00Z', 
        'end': '2021-09-01T00:00:00Z', 
        'flightDirection': 'A',
        'processingLevel': 'SLC',
        'frame': 84}

In [40]:
# search for scenes
results = asf.geo_search(intersectsWith=aoi, **opts)
print(f'{len(results)} scenes found')

2022-03-05 17:40:03,314 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): api.daac.asf.alaska.edu:443
2022-03-05 17:40:05,361 - urllib3.connectionpool - DEBUG - https://api.daac.asf.alaska.edu:443 "POST /services/search/param HTTP/1.1" 200 None
3 scenes found


In [ ]:
# Download scenes
%%time 

#Each scene is ~4 GB
results.download(path = '/tmp', session=user_pass_session)

2022-03-05 17:54:46,838 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): datapool.asf.alaska.edu:443
2022-03-05 17:54:47,985 - urllib3.connectionpool - DEBUG - https://datapool.asf.alaska.edu:443 "GET /SLC/SA/S1A_IW_SLC__1SDV_20210830T121356_20210830T121423_039459_04A972_A57E.zip HTTP/1.1" 307 428
2022-03-05 17:54:47,990 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): sentinel1.asf.alaska.edu:443
2022-03-05 17:54:49,671 - urllib3.connectionpool - DEBUG - https://sentinel1.asf.alaska.edu:443 "GET /SLC/SA/S1A_IW_SLC__1SDV_20210830T121356_20210830T121423_039459_04A972_A57E.zip HTTP/1.1" 303 0
2022-03-05 17:54:49,678 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): dy4owt9f80bz7.cloudfront.net:443
2022-03-05 17:54:51,227 - urllib3.connectionpool - DEBUG - https://dy4owt9f80bz7.cloudfront.net:443 "GET /s3-06b3213905ebfa9144871d6c7f9306f0/asf-ngap2w-p-s1-slc-7b420b89.s3.us-west-2.amazonaws.com/S1A_IW_SLC__1SDV_20210830T121356_20